In [2]:
!pip install pykrx

In [3]:
from pykrx import stock

In [7]:
# KOSPI 
stock_code = stock.get_market_ticker_list(market="KOSPI")
(stock_code[:6], len(stock_code))

# 937개의 종목 있음

(['095570', '006840', '027410', '282330', '138930', '001460'], 937)

In [8]:
# 종목명 반환
stock_name = stock.get_market_ticker_name("005930")
stock_name

'삼성전자'

In [30]:
from datetime import date, timedelta

today = date.today().strftime('%Y%m%d')
yesterday = (date.today() - timedelta(1)).strftime('%Y%m%d')

# 거래량

In [54]:
# 개별종목 시세 추이

df_volume = stock.get_market_ohlcv_by_date(fromdate=yesterday, todate=today, ticker="005930")
df_volume

,시가,고가,저가,종가,거래량
날짜,,,,,
2021-09-17,76300,76700,75900,76700,4147816


In [55]:
#df_volume.columns
df_volume = df_volume[['거래량']]
df_volume

,거래량
날짜,
2021-09-17,4147816


# PER, PBR

In [41]:
# PER, PBR

df_per = stock.get_market_fundamental_by_date(today, today, "005930")
df_per

,BPS,PER,PBR,EPS,DIV,DPS
날짜,,,,,,
2021-09-17,39406,19.79,1.93,3841,3.94,2994


In [56]:
df_per = df_per[['PER', 'PBR']]
df_per

,PER,PBR
날짜,,
2021-09-17,19.79,1.93


# 거래실적

In [73]:
# 투자자별 거래실적

df_value = stock.get_market_trading_value_by_date(yesterday, yesterday, "005930")
df_value


,기관합계,기타법인,개인,외국인합계,전체
날짜,,,,,
2021-09-16,-169972285200,2789255200,131008255500,36174774500,0


In [74]:
df_value.drop(['전체'], axis = 1)

,기관합계,기타법인,개인,외국인합계
날짜,,,,
2021-09-16,-169972285200,2789255200,131008255500,36174774500


# 합치기

In [75]:
import pandas as pd

df = pd.concat([df_volume, df_per, df_value],axis=1)
df

,거래량,PER,PBR,기관합계,기타법인,개인,외국인합계,전체
날짜,,,,,,,,
2021-09-16,NaN,NaN,NaN,-1.699723e+11,2.789255e+09,1.310083e+11,3.617477e+10,0.0
2021-09-17,4147816.0,19.79,1.93,NaN,NaN,NaN,NaN,NaN


# 함수화

In [10]:
# !pip install pykrx
from pykrx import stock
import pandas as pd
from datetime import date, timedelta

class KRX:
    def getDate(self):
        self.today = date.today().strftime('%Y%m%d')
        self.yesterday = (date.today() - timedelta(1)).strftime('%Y%m%d')
        
    def getVolume(self, code):
        self.volume = stock.get_market_ohlcv_by_date(self.yesterday, self.today, code)
        self.volume = self.volume[['거래량']]
    
    def getPER(self, code):
        self.per = stock.get_market_fundamental_by_date(self.today, self.today, code)
        self.per = self.per[['PER', 'PBR']]
    
    def getValue(self, code):
        self.value = stock.get_market_trading_value_by_date(self.today, self.today, code)
        self.value.drop(['전체'], axis = 1)
    
    def saving(self, code):
        self.df = pd.concat([self.volume, self.per, self.value], axis=1)
        self.stockname = stock.get_market_ticker_name(code)
        self.df.to_excel("./data/update" + self.stockname +".xlsx")
        
        
if __name__ == "__main__":
    code = "005930" # 종목코드 
    krx = KRX()
    krx.getDate()
    krx.getVolume(code)
    krx.getPER(code)
    krx.getValue(code)
    krx.saving(code)
    
